# 케라스

In [1]:
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation
from keras.models import Model

inputs = Input(shape = (28,28,1))
x = Conv2D(filters = 32, kernel_size = (3,3), strides = (1,1), padding = 'same')(inputs)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2,2), strides = (2,2))(x)
x = Dropout(0.5)(x)
x = Conv2D(filters = 64, kernel_size = (3,3), strides = (1,1), padding = 'same')(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2,2), strides = (2,2))(x)
x = Dropout(0.5)(x)
x = Flatten()(x)
x = Dense(128)(x)
outputs = Dense(10, activation = 'softmax')(x)

model = Model(inputs = inputs, outputs = outputs)

model.summary()

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
___________________________________________

# 파이토치

In [2]:
import torch
import torch.nn as nn

class Standard(nn.Module):
    
    def __init__(self):
        super(Standard, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
        self.relu1 = nn.ReLU(inplace = True) # relu를 통과한 값이 갱신되도록 해주는게 inplace=True
        self.maxpool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 1)
        self.relu2 = nn.ReLU(inplace = True)
        self.maxpool2 = nn.MaxPool2d(kernel_size = 2, stride = 2) # 여기까지 왔을 때의 output이 7*7*64=3136
        self.dense1 = nn.Linear(in_features = 3136, out_features = 128)
        self.relu3 = nn.ReLU(inplace = True)
        self.dense2 = nn.Linear(in_features = 128, out_features = 10)
        self.relu4 = nn.ReLU(inplace = True)
        
    def forward(self, x):
        
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.maxpool1(out)
        out = self.conv2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)
        out = out.view(out.size(0),-1) # view는 토치에서 reshape 기능, 그래서 Flatten으로 활용
        out = self.dense1(out)
        out = self.relu3(out)
        out = self.dense2(out)
        out = self.relu4(out)
        
        return out
    
if __name__ == '__main__':

    from torchsummary import summary
    model = Standard()
    summary(model, (1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
              ReLU-2           [-1, 32, 28, 28]               0
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 64, 14, 14]          18,496
              ReLU-5           [-1, 64, 14, 14]               0
         MaxPool2d-6             [-1, 64, 7, 7]               0
            Linear-7                  [-1, 128]         401,536
              ReLU-8                  [-1, 128]               0
            Linear-9                   [-1, 10]           1,290
             ReLU-10                   [-1, 10]               0
Total params: 421,642
Trainable params: 421,642
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.65
Params size (MB): 1.61
Estimated T

In [ ]:
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import torch.optim as optim

#  GPU를 사용 가능하다면 device 값이 cuda가 되고, 아니라면 cpu

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정

torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정

if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# 학습에 사용할 파라미터를 설정

learning_rate = 0.001
training_epochs = 15
batch_size = 100

# 데이터셋을 정의

mnist_train = MNIST(root='MNIST_data/', # 다운로드 경로 지정
                    train=True, # True로 지정하면 훈련 데이터로 다운로드
                    transform=ToTensor(), # 텐서로 변환
                    download=True)

mnist_test = MNIST(root='MNIST_data/', 
                   train=False, # False로 지정하면 훈련 데이터로 다운로드
                   transform=ToTensor(), 
                   download=True)

# 데이터 로더를 사용하여 배치 크기 지정

data_loader = DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)

# 모델 정의

model = Standard().to(device)

# loss function과 optimizer 정의 (크로스 엔트로피, Adam)

loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 총 배치 수는 600이고 배치 크기가 100이므로 훈련 데이터는 60000개

total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

# 모델 훈련 과정

for epoch in range(training_epochs):
    
    avg_loss = 0
    
    # X는 mnist_train.train_data[i] 즉 미니 배치 
    # Y는 mnist_train.train_labels[i] 즉 레이블
    
    for X, Y in data_loader: 
        
        X = X.to(device)
        
        y_true = Y.to(device)
        y_pred = model(X)
        
        loss = loss_fn(y_pred, y_true)
        
        optimizer.zero_grad() # gradient를 0으로 초기화
        
        loss.backward() # loss를 역전파
        
        optimizer.step() # 역전파된 loss를 가지고 weights를 갱신
        
        avg_loss += loss / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_loss))
    
# 모델 테스트 과정

with torch.no_grad():
    
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    acc = correct_prediction.float().mean()
    print('Accuracy:', acc.item())